In [1]:
import os
import numpy as np
import trimesh
from scipy.ndimage import zoom
from tqdm import tqdm
!pip install kagglehub
import kagglehub

In [2]:
# Download latest version
path = kagglehub.dataset_download("balraj98/modelnet40-princeton-3d-object-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\amirh\.cache\kagglehub\datasets\balraj98\modelnet40-princeton-3d-object-dataset\versions\1


In [5]:
dataset_root = os.path.join(path, "ModelNet40")  # ⚠️ مطمئن شو path تعریف شده
selected_classes = [
    'airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car',
    'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot',
    'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor',
    'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
    'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase',
    'wardrobe', 'xbox'
]
voxel_dim = 32  # اندازه شبکه وکسل
num_orientations = 12  # Number of rotation steps (e.g., every 30 degrees)

In [7]:
import trimesh
import numpy as np

def mesh_to_pointcloud(path_to_off_file, num_points=1024):
    mesh = trimesh.load(path_to_off_file)
    if not isinstance(mesh, trimesh.Trimesh):
        mesh = mesh.dump().sum()
    
    # Sample points uniformly from surface
    points, _ = trimesh.sample.sample_surface(mesh, num_points)
    
    return points.astype(np.float32)  # shape: (num_points, 3)

In [11]:
def rotate_pointcloud(pc, angle_deg, axis='y'):
    angle_rad = np.deg2rad(angle_deg)
    if axis == 'y':
        R = np.array([
            [np.cos(angle_rad), 0, np.sin(angle_rad)],
            [0, 1, 0],
            [-np.sin(angle_rad), 0, np.cos(angle_rad)]
        ])
    else:
        raise NotImplementedError("Only Y-axis rotation supported")
    return pc @ R.T


def load_pointcloud_dataset_with_orientation(dataset_path, num_points=1024, selected_classes=None, split='train', num_orientations=8):
    X, y_class, y_orientation = [], [], []

    if selected_classes is None:
        selected_classes = sorted(os.listdir(dataset_path))
    
    label_map = {cls_name: idx for idx, cls_name in enumerate(selected_classes)}

    for cls_name in tqdm(selected_classes, desc="Classes"):
        class_path = os.path.join(dataset_path, cls_name, split)
        if not os.path.exists(class_path):
            continue
        
        file_list = [f for f in os.listdir(class_path) if f.endswith('.off')]
        for file_name in tqdm(file_list, desc=f"{cls_name} ({split})", leave=False):
            file_path = os.path.join(class_path, file_name)
            try:
                base_pc = mesh_to_pointcloud(file_path, num_points)
                base_pc -= np.mean(base_pc, axis=0)  # normalize center
                base_pc /= np.max(np.linalg.norm(base_pc, axis=1))  # normalize scale

                for ori_idx in range(num_orientations):
                    angle = (360 / num_orientations) * ori_idx
                    rotated_pc = rotate_pointcloud(base_pc, angle, axis='y')

                    X.append(rotated_pc)
                    y_class.append(label_map[cls_name])
                    y_orientation.append(ori_idx)
            except Exception as e:
                print(f"⚠️ Error in {file_path}: {e}")
    
    X = np.array(X, dtype=np.float32)
    y_class = np.array(y_class, dtype=np.int32)
    y_orientation = np.array(y_orientation, dtype=np.int32)
    return X, y_class, y_orientation, label_map

In [13]:
X_pc, y_class_pc, y_ori_pc, label_map = load_pointcloud_dataset_with_orientation(
    dataset_path=dataset_root,
    num_points=1024,
    selected_classes=selected_classes,
    split='train',
    num_orientations=12
)

curtain (train):  47%|████▋     | 65/138 [00:09<00:28,  2.52it/s]C:\Users\amirh\anaconda3\Lib\site-packages\trimesh\grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)

Classes: 100%|██████████| 40/40 [26:02<00:00, 39.07s/it]       


In [15]:
np.save("MN40_X_pc_oriente.npy", X_pc)
np.save("MN40_y_class_pc_oriente.npy", y_class_pc)
np.save("MN40_y_orientation_pc_oriente.npy", y_ori_pc)